In [ ]:
!pip install plotly

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import timedelta
from scipy.stats import norm
from IPython.display import display, clear_output
import ipywidgets as widgets
from google.colab import files
import io

# Global state
df = None
dropdown_mapping = {}

# Upload and dropdown widgets
upload_button = widgets.FileUpload(accept='.xlsx', multiple=False, description='Upload Excel File')
dropdown = widgets.Dropdown(options=[], description='Select Task:', disabled=True)
search_box = widgets.Text(placeholder='Search task name...', description='Search:')
output = widgets.Output()

# Functions
def handle_upload(uploaded_file):
    global df, dropdown_mapping

    try:
        df = pd.read_excel(io.BytesIO(uploaded_file))
        df.fillna("", inplace=True)

        df["START DATE"] = pd.to_datetime(df["START DATE"], errors='coerce')
        df["FINISH DATE"] = pd.to_datetime(df["FINISH DATE"], errors='coerce')

        PLANNED_COL = "% PLANNED"
        ACTUAL_COL = "% ACTUAL"
        ACTIVITY_COL = "TASK ID"
        SUMMARY_COL = "IS SUMMARY"
        OUTLINE_COL = "OUTLINE NUMBER"
        OUTLINE_LEVEL_COL = "OUTLINE LEVEL"
        DURATION_COL = "DURATION (DAYS)"

        main_activities = df[(df[SUMMARY_COL] == True) & (df[ACTIVITY_COL] != "")].copy()
        main_activities["DISPLAY_NAME"] = main_activities[ACTIVITY_COL].astype(str) + " - " + main_activities["NAME"]
        dropdown_mapping.clear()
        dropdown_mapping.update(dict(zip(main_activities["DISPLAY_NAME"], main_activities[ACTIVITY_COL])))

        dropdown.options = list(dropdown_mapping.keys())
        dropdown.disabled = False
        search_box.value = ""
        output.clear_output()
        with output:
            print("File successfully loaded! Use the search box and select a task.")
    except Exception as e:
        with output:
            clear_output()
            print(f"Error loading file: {str(e)}")

def get_sub_activities(main_outline):
    return df[(df["OUTLINE NUMBER"].astype(str).str.startswith(str(main_outline))) & (df["IS SUMMARY"] != True)]

def generate_s_curve(days, total_progress):
    x = np.linspace(-3, 3, days)
    y = norm.cdf(x)
    return y * total_progress

def plot_activity_s_curve_display(display_name):
    ACTIVITY_COL = "TASK ID"
    PLANNED_COL = "% PLANNED"
    ACTUAL_COL = "% ACTUAL"
    SUMMARY_COL = "IS SUMMARY"
    OUTLINE_COL = "OUTLINE NUMBER"
    DURATION_COL = "DURATION (DAYS)"

    task_id = dropdown_mapping[display_name]
    activity = df[(df[ACTIVITY_COL] == task_id) & (df[SUMMARY_COL] == True)].iloc[0]

    start = activity["START DATE"]
    finish = activity["FINISH DATE"]
    duration = activity[DURATION_COL]
    days_total = (finish - start).days + 1
    today = pd.Timestamp.today().normalize()
    planned_progress = activity[PLANNED_COL]
    actual_progress = activity[ACTUAL_COL]
    remaining_progress = max(0.0, planned_progress - actual_progress)

    estimated_days_remaining = remaining_progress * duration
    estimated_finish_date = today + timedelta(days=round(estimated_days_remaining)) if estimated_days_remaining > 0 else today

    full_timeline = pd.date_range(start, max(finish, today))
    days_extended = (full_timeline[-1] - start).days + 1

    planned_s_curve = generate_s_curve(days_total, planned_progress)
    actual_s_curve = generate_s_curve(days_total, actual_progress)

    planned_s_curve_full = np.concatenate([
        planned_s_curve,
        np.full(days_extended - days_total, planned_s_curve[-1])
    ])

    actual_s_curve_full = np.concatenate([
        actual_s_curve,
        np.full(days_extended - days_total, actual_s_curve[-1])
    ])

    current_day = (today - start).days
    current_progress = actual_s_curve_full[current_day] if 0 <= current_day < len(actual_s_curve_full) else actual_s_curve_full[-1]

    future_days = 7
    pred_timeline = pd.date_range(today, periods=future_days + 1)
    pred_s_curve = generate_s_curve(future_days + 1, 1)
    pred_progress = current_progress + (1 - current_progress) * pred_s_curve

    extended_planned_s_curve = np.concatenate([
        planned_s_curve_full,
        np.full(len(pred_timeline), planned_s_curve_full[-1])
    ])

    # Plot
    plt.figure(figsize=(14, 7))
    plt.plot(full_timeline, planned_s_curve_full, label="Planned", color="blue", linewidth=2)
    plt.plot(full_timeline, actual_s_curve_full, label="Actual", color="green", linewidth=2)

    if current_day >= 0:
        plt.fill_between(full_timeline[:current_day+1], planned_s_curve_full[:current_day+1],
                         actual_s_curve_full[:current_day+1], color="lightgreen", alpha=0.5,
                         label="Progress Gap")

    plt.axvline(today, color="red", linestyle=":", label=f"Today ({today.date()})")
    plt.plot(pred_timeline, pred_progress, 'r--', linewidth=2, label="Predicted to 100% (7-day)")
    plt.fill_between(pred_timeline, extended_planned_s_curve[-len(pred_timeline):],
                     pred_progress, color="lightcoral", alpha=0.5, label="Prediction Gap")
    plt.plot(pred_timeline, extended_planned_s_curve[-len(pred_timeline):], 'b--', linewidth=2,
             label="Extended Planned")

    plt.axvline(finish, color="purple", linestyle="-.", label=f"Planned Finish ({finish.date()})")
    plt.title(f"S-Curve Progress: {activity['NAME']} ({task_id})")
    plt.xlabel("Date")
    plt.ylabel("Progress (0 to 1)")
    plt.legend(loc='upper left', bbox_to_anchor=(1, 1))
    plt.grid(True)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

    # Effort Estimation
    print(f"\n--- Effort Estimation for '{activity['NAME']}' ({task_id}) ---")
    print(f"Start Date: {start.date()}")
    print(f"Planned Finish Date: {finish.date()}")
    print(f"Today's Date: {today.date()}")
    print(f"Planned Progress: {planned_progress:.2%}")
    print(f"Actual Progress: {actual_progress:.2%}")
    print(f"Remaining Progress: {remaining_progress:.2%}")
    print(f"Duration (Days): {duration}")
    print(f"Estimated Days Remaining: {estimated_days_remaining:.2f}")
    print(f"Estimated Finish Date: {estimated_finish_date.date()}")

    sub_activities = get_sub_activities(activity[OUTLINE_COL])
    sub_activities = sub_activities[sub_activities[ACTUAL_COL] < 1]

    slow_subs = sub_activities[
        (sub_activities[PLANNED_COL] > 0) &
        (sub_activities[ACTUAL_COL] < sub_activities[PLANNED_COL])
    ]
    delayed_subs = sub_activities[
        (sub_activities["START DATE"] < today) & (sub_activities[PLANNED_COL] > 0) &
        (sub_activities[ACTUAL_COL] == 0)
    ]
    not_started = sub_activities[(sub_activities[PLANNED_COL] == 0) & (sub_activities[ACTUAL_COL] == 0)]

    def display_subs(title, subs):
        if subs.empty:
            print(f"\n{title}: None")
        else:
            print(f"\n{title}:")
            for _, row in subs.iterrows():
                print(f"- {row['TASK ID']} | {row['NAME']} | Planned: {row[PLANNED_COL]:.0%} | Actual: {row[ACTUAL_COL]:.0%}")

    display_subs("🔴 Delayed", delayed_subs)
    display_subs("🟡 Slow Progress", slow_subs)
    display_subs("⚪ Not Started", not_started)

# Event handlers
def on_upload_change(change):
    if upload_button.value:
        uploaded_file = next(iter(upload_button.value.values()))
        handle_upload(uploaded_file['content'])

def on_dropdown_change(change):
    if change['new'] in dropdown_mapping:
        with output:
            clear_output()
            plot_activity_s_curve_display(change['new'])

def on_search_change(change):
    search = search_box.value.lower()
    filtered = [key for key in dropdown_mapping if search in key.lower()]
    dropdown.options = filtered if filtered else ["No match found"]

# Bind observers
upload_button.observe(on_upload_change, names='value')
dropdown.observe(on_dropdown_change, names='value')
search_box.observe(on_search_change, names='value')

# Display UI
display(upload_button)
display(search_box)
display(dropdown)
display(output)